In [ ]:
import numpy as np
from scipy.optimize import minimize
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import clear_output
import time
import plotly.graph_objs as goa
import matplotlib as mpl
import torch.nn.functional as F
import random
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import copy
from itertools import combinations
from torch.optim.lr_scheduler import _LRScheduler
from fancy_einsum import einsum
from scipy.spatial import ConvexHull
import os
import examples_setup as utils
mpl.style.use('seaborn-v0_8')
mpl.rcParams['figure.figsize'] = (15,10)
fontsize = 20
mpl.rcParams['font.size'] = fontsize
mpl.rcParams['xtick.labelsize'] = fontsize
mpl.rcParams['ytick.labelsize'] = fontsize
mpl.rcParams['legend.fontsize'] = fontsize
mpl.rcParams['axes.titlesize'] = fontsize
mpl.rcParams['axes.labelsize'] = fontsize

In [ ]:
# your chosen seed
chosen_seed = 100
utils.set_seed(chosen_seed)
#Checking for errors
lr_print_rate = 0
# Configure the hyperparameters
f = 40
k = 1
n = 2
MSE = True #else Crossentropy
nonlinearity = F.relu
tied = True
final_bias = False
hidden_bias = False
unit_weights = False
learnable_scale_factor = False
initial_scale_factor = 1# (1/(1-np.cos(2*np.pi/f)))**0.5
standard_magnitude = False
initial_embed = None
initial_bias = None
epochs = 10000
logging_loss = True
#Scheduler params
max_lr = 2
initial_lr = 0.02
warmup_frac = 0.05
final_lr = 0.5
decay_factor=(final_lr/max_lr)**(1/(epochs * (1-warmup_frac)))
warmup_steps = int(epochs * warmup_frac)
store_rate = epochs//100
plot_rate=0 #epochs/5
# Instantiate synthetic dataset
dataset = utils.SyntheticKHot(f,k)
batch_size = len(dataset) #Full batch gradient descent
loader = DataLoader(dataset, batch_size=batch_size, shuffle = True, num_workers=0)

#Define the Loss function
criterion = nn.MSELoss() if MSE else nn.CrossEntropyLoss() 

# Instantiate the model
# initial_embed = torch.tensor(np.array([1/(1-np.cos(2*np.pi/f))**0.5*np.array([np.cos(2*np.pi*i/f),np.sin(2*np.pi*i/f)]) for i in range(f)]),dtype=torch.float32).T * 0.5
# initial_bias = -torch.ones(f)*(1/(1-np.cos(2*np.pi/f))- 1)*0.25
model = utils.Net(f, n,
            tied = tied,
            final_bias = final_bias,
            hidden_bias = hidden_bias,
            nonlinearity=nonlinearity,
            unit_weights=unit_weights,
            learnable_scale_factor=learnable_scale_factor,
            standard_magnitude=standard_magnitude,
            initial_scale_factor = initial_scale_factor,
            initial_embed = initial_embed,
            initial_bias = initial_bias)

# Define loss function and optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=initial_lr)

#Define a learning rate schedule
scheduler = utils.CustomScheduler(optimizer, warmup_steps, max_lr, decay_factor)

# Train the model
losses, weights_history, model_history = utils.train(model, loader, criterion, optimizer, epochs, logging_loss, plot_rate, store_rate, scheduler, lr_print_rate)

In [ ]:
utils.plot_weights_interactive(weights_history, store_rate=store_rate)

In [ ]:
# your chosen seed
chosen_seed = 101
new_run = True
utils.set_seed(chosen_seed)
#Checking for errors
lr_print_rate = 0
# Configure the hyperparameters
f = 250
k = 1
n = 9
MSE = True #else Crossentropy
nonlinearity = F.relu
tied = True
final_bias = False
hidden_bias = False
unit_weights = False
learnable_scale_factor = False
initial_scale_factor = 1# (1/(1-np.cos(2*np.pi/f)))**0.5
standard_magnitude = False
initial_embed = None
initial_bias = None
epochs = 1000000
logging_loss = True
#Scheduler params
max_lr = 8
initial_lr = 0.02
warmup_frac = 0.05
final_lr = 1
decay_factor=(final_lr/max_lr)**(1/(epochs * (1-warmup_frac)))
warmup_steps = int(epochs * warmup_frac)
store_rate = epochs//100
plot_rate=0 #epochs/5
# Instantiate synthetic dataset
dataset = utils.SyntheticKHot(f,k)
batch_size = len(dataset) #Full batch gradient descent
loader = DataLoader(dataset, batch_size=batch_size, shuffle = True, num_workers=0)

#Define the Loss function
criterion = nn.MSELoss() if MSE else nn.CrossEntropyLoss() 

# Instantiate the model
# initial_embed = torch.tensor(np.array([1/(1-np.cos(2*np.pi/f))**0.5*np.array([np.cos(2*np.pi*i/f),np.sin(2*np.pi*i/f)]) for i in range(f)]),dtype=torch.float32).T * 0.5
# initial_bias = -torch.ones(f)*(1/(1-np.cos(2*np.pi/f))- 1)*0.25
if new_run:
    model = utils.Net(f, n,
                tied = tied,
                final_bias = final_bias,
                hidden_bias = hidden_bias,
                nonlinearity=nonlinearity,
                unit_weights=unit_weights,
                learnable_scale_factor=learnable_scale_factor,
                standard_magnitude=standard_magnitude,
                initial_scale_factor = initial_scale_factor,
                initial_embed = initial_embed,
                initial_bias = initial_bias)

# Define loss function and optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=initial_lr)

#Define a learning rate schedule
scheduler = utils.CustomScheduler(optimizer, warmup_steps, max_lr, decay_factor)

# Train the model
if new_run:
    losses, weights_history, model_history = utils.train(model, loader, criterion, optimizer, epochs, logging_loss, plot_rate, store_rate, scheduler, lr_print_rate)
else:
    new_losses, new_weights_history, new_model_history = utils.train(model, loader, criterion, optimizer, epochs, logging_loss, plot_rate, store_rate, scheduler, lr_print_rate)
    losses += new_losses
    weights_history += new_weights_history
    model_history += new_model_history
    

In [ ]:
plt.plot(losses)

In [ ]:
utils.plot_weights_interactive(weights_history, store_rate=store_rate)

In [ ]:
groups, directions = utils.group_vectors(model.unembedding.weight.data.detach().numpy(), 0.001)
print(len(groups))

In [ ]:
model.embedding.weight.data.detach().pow(2).sum(dim=1)

In [ ]:
pca_8

In [ ]:
n_groups = {}
models = {}
for n in range(2,11):
    print(f'current n: {n}')
    n_groups[n] = []
    models[n] = []
    for chosen_seed in range(1,6):
        print(f'current seed: {chosen_seed}')
        utils.set_seed(chosen_seed)
        #Checking for errors
        lr_print_rate = 0
        # Configure the hyperparameters
        f = 100
        k = 1
        MSE = True #else Crossentropy
        nonlinearity = F.relu
        tied = True
        final_bias = False
        hidden_bias = False
        unit_weights = False
        learnable_scale_factor = False
        initial_scale_factor = 1# (1/(1-np.cos(2*np.pi/f)))**0.5
        standard_magnitude = False
        initial_embed = None
        initial_bias = None
        epochs = 40000
        logging_loss = True
        #Scheduler params
        max_lr = 3
        initial_lr = 0.02
        warmup_frac = 0.05
        final_lr = 1
        decay_factor=(final_lr/max_lr)**(1/(epochs * (1-warmup_frac)))
        warmup_steps = int(epochs * warmup_frac)
        store_rate = epochs//100
        plot_rate=0 #epochs/5
        # Instantiate synthetic dataset
        dataset = utils.SyntheticKHot(f,k)
        batch_size = len(dataset) #Full batch gradient descent
        loader = DataLoader(dataset, batch_size=batch_size, shuffle = True, num_workers=0)

        #Define the Loss function
        criterion = nn.MSELoss() if MSE else nn.CrossEntropyLoss() 

        # Instantiate the model
        # initial_embed = torch.tensor(np.array([1/(1-np.cos(2*np.pi/f))**0.5*np.array([np.cos(2*np.pi*i/f),np.sin(2*np.pi*i/f)]) for i in range(f)]),dtype=torch.float32).T * 0.5
        # initial_bias = -torch.ones(f)*(1/(1-np.cos(2*np.pi/f))- 1)*0.25
        model = utils.Net(f, n,
                    tied = tied,
                    final_bias = final_bias,
                    hidden_bias = hidden_bias,
                    nonlinearity=nonlinearity,
                    unit_weights=unit_weights,
                    learnable_scale_factor=learnable_scale_factor,
                    standard_magnitude=standard_magnitude,
                    initial_scale_factor = initial_scale_factor,
                    initial_embed = initial_embed,
                    initial_bias = initial_bias)

        # Define loss function and optimizer
        optimizer = torch.optim.SGD(model.parameters(), lr=initial_lr)

        #Define a learning rate schedule
        scheduler = utils.CustomScheduler(optimizer, warmup_steps, max_lr, decay_factor)

        # Train the model
        losses, weights_history, model_history = utils.train(model, loader, criterion, optimizer, epochs, logging_loss, plot_rate, store_rate, scheduler, lr_print_rate)
        groups, directions = utils.group_vectors(model.unembedding.weight.data.detach().numpy(), 0.001)
        n_groups[n].append(len(groups))
        models[n].append(model)



In [ ]:
n_groups

In [ ]:
n_groups3 = {}
for n in models.keys():
        n_groups2[n] = []
        for seed in range(len(models[n])):
                groups, directions = utils.group_vectors(models[n][seed].unembedding.weight.data.detach().numpy(), 0.1)
                n_groups2[n].append(len(groups))

In [ ]:
n_groups2

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

# Prepare your data for the linear regression model
x = np.array(list(n_groups2.keys()))
y = np.array([np.mean(val) for val in n_groups2.values()])

# Reshape your data since it has a single feature
x = x.reshape((-1, 1))

# Apply linear regression
model = LinearRegression()
model.fit(x, y)

# The line of best fit is represented as: y = mx + c
m = model.coef_[0]
c = model.intercept_

print(f"The line of best fit is: y = {m}x + {c}")
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
import numpy as np

x = np.array(list(n_groups2.keys())).reshape((-1, 1))
y = np.array([np.mean(val) for val in n_groups2.values()])

# Create a model with polynomial features
model = make_pipeline(PolynomialFeatures(2), LinearRegression())
model.fit(x, y)

# The coefficients of the quadratic model (quadratic term, linear term, and intercept)
coeff = model.named_steps['linearregression'].coef_
intercept = model.named_steps['linearregression'].intercept_

print(f'The equation of best fit is: y = {coeff[2]}x^2 + {coeff[1]}x + {intercept}')



In [ ]:
xs = np.linspace(1,10,500)
for n in range(2,11):
    plt.scatter([n]*5, n_groups2[n], color='blue')
plt.plot(xs, xs * 7.52-12.853)
plt.xlabel('Dimension')
plt.ylabel('Number of Features Encoded')
plt.show()

In [ ]:
n_groups2 = {}
for n in models.keys():
        n_groups2[n] = []
        for seed in range(len(models[n])):
                groups, directions = utils.group_vectors(models[n][seed].unembedding.weight.data.detach().numpy(), 0.01)
                n_groups2[n].append(len(groups))

In [ ]:
weights = {}
n = 5
weights['embedding.weight'] = [m.embedding.weight for m in models[n]]
weights['unembedding.weight'] = [m.unembedding.weight for m in models[n]]
weights['unembedding.bias'] = [m.unembedding.bias for m in models[n]]

utils.plot_weights_interactive(weights)